---
<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/image_igac.jpg" alt="Imagen_IGAC" width="280">
</p>

---

# ***Fundamentos de LLMs con Python: Explorando ChatGPT y LangChain***

---

#### ***Instructor: [Luis Andrés Campos Maldonado](https://www.linkedin.com/in/lacamposm/)***

##### ***Email: luisandres.campos@igac.gov.co***

##### ***Contratista-Observatorio Inmobiliario Catastral***





---

# ***Clase 07 - 12 de abril de 2024***
---

## ***Implementando memoria en Chatbots con LangChain y Streamlit***

**Objetivos de Aprendizaje:**

- Introducir el concepto de memoria en las interacciones con modelos de lenguaje como ChatGPT.
- Demostrar cómo implementar y gestionar memoria en un chatbot.
- Utilizar [Streamlit](https://streamlit.io/) para crear una interfaz de usuario simple para el chatbot, enfocándose en la funcionalidad de memoria.

## ***Concepto de Memoria en LangChain***

Un componente esencial en muchas aplicaciones basadas en inteligencia artificial generativa, es que tienen interfaces conversacionales, es decir, tiene la capacidad de referirse a información introducida anteriormente en la conversación. A esto, en LangChain, se le llama "memory" (memoria).

###  ***Introducción a la Memoria en LangChain***

La memoria en LangChain permite a un sistema conversacional acceder a una ventana de mensajes pasados directamente, o incluso mantener un modelo del mundo que actualiza constantemente, lo cual le permite mantener información sobre entidades y sus relaciones. Esto se logra a través de dos acciones básicas: leer (READ) y escribir (WRITE).


### ***Cómo Funciona la Memoria***

1. **Lectura (READ):** Antes de ejecutar la lógica central, el sistema lee de su sistema de memoria y complementa las entradas de usuario.
2. **Escritura (WRITE):** Después de ejecutar la lógica central pero antes de devolver la respuesta, el sistema escribe las entradas y salidas del ciclo actual en la memoria, para que puedan ser referenciadas en ciclos futuros.


<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/memory_langchain.png" alt="Imagen_IGAC" width="900" height="450">
</p>

In [1]:
%%capture
!pip install pandas openpyxl langchain openai langchain-openai langchain-community langchain-core langchain-text-splitters chromadb pypdf

### ***Construyendo Memoria en un Sistema***

La construcción de la memoria en un sistema implica dos decisiones de diseño clave:

- **Cómo se almacena el estado:** LangChain ofrece integraciones para almacenar mensajes de chat, desde listas en memoria hasta bases de datos persistentes.
- **Cómo se consulta el estado:** Sobre los mensajes de chat almacenados, se pueden construir estructuras de datos y algoritmos que ofrezcan una vista útil de esos mensajes, desde sistemas simples que solo retornan los mensajes más recientes hasta sistemas más sofisticados que extraen y retornan información sobre entidades específicas.

#### ***Ejemplo Práctico con `ConversationBufferMemory`***

`ConversationBufferMemory` en LangChain representa una implementación sencilla y directa de la gestión de memoria en aplicaciones conversacionales. Este tipo de memoria se centra en almacenar una lista de mensajes de chat en un búfer, que luego se pueden utilizar para influir en las respuestas generadas por modelos de lenguaje. Su funcionalidad clave radica en su capacidad para retener un registro de la interacción entre el usuario y el sistema, proporcionando un contexto conversacional que mejora la coherencia y relevancia de las respuestas del sistema. La utilización de esta herramienta es particularmente beneficiosa en escenarios donde la continuidad de la conversación y la relevancia contextual son cruciales para la experiencia del usuario. En bots de servicio al cliente, la capacidad de referirse a interacciones anteriores en tiempo real mejora significativamente la interacción usuario-máquina.

- ***Asistente de Recomendación de Películas:***

El usuario interactuará con un asistente que le recomienda películas basándose en las preferencias expresadas en la conversación. La memoria del asistente recordará los géneros de películas mencionados anteriormente para ajustar sus recomendaciones.

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

# El usuario saluda y muestra interés en una recomendación, agregamos mensajes.
memory.chat_memory.add_user_message("Hola, ¿me recomiendas una película?")
memory.chat_memory.add_ai_message("¡Claro! ¿Qué género prefieres?")
# El usuario responde con su preferencia.
memory.chat_memory.add_user_message("Me gustan las películas de ciencia ficción.")
# Aquí, 'history' se carga desde la memoria y se espera que el cadena (y probablemente el promtp) 
# acepte una entrada con este nombre.
memory.load_memory_variables({})

{'history': 'Human: Hola, ¿me recomiendas una película?\nAI: ¡Claro! ¿Qué género prefieres?\nHuman: Me gustan las películas de ciencia ficción.'}

Basándonos en el historial almacenado, el sistema podría generar una recomendación personalizada. Aunque en este ejemplo no integraremos un LLM (e.g ChatOpenAI), podemos simular una lógica de respuesta:

In [3]:
# Cargamos las variables de memoria antes de generar la siguiente respuesta
memory_variables = memory.load_memory_variables({})

if "ciencia ficción" in memory_variables["history"]:
    print("Te recomiendo ver 'Interstellar', es una gran película de ciencia ficción.")
else:
    print("¿Tienes algún género favorito?")

Te recomiendo ver 'Interstellar', es una gran película de ciencia ficción.


#### ***Ejemplo: Parámetros con `ConversationBufferMemory`***

`ConversationBufferMemory` ofrece opciones configurables a través de sus parámetros, permitiendo adaptar el comportamiento de la memoria a las necesidades específicas de la aplicación.
- ***memory_key***: Este parámetro determina la clave bajo la cual se almacenarán y recuperarán las variables de memoria. Es fundamental para asegurar que los datos de memoria se sincronicen correctamente con los prompts del modelo de lenguaje, facilitando así la generación de respuestas contextualizadas.

- ***return_messages***: Este parámetro controla el formato en el que se retornan los mensajes almacenados en la memoria. Al configurarlo como True, se indica que los mensajes deben retornarse como una lista de objetos ChatMessage, en lugar de un único string concatenado.

In [4]:
# "chat_history" como key. El historial de chat será accesible bajo esta denominación para su uso en la generación de prompts.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

memory.chat_memory.add_user_message("Hola, ¿me recomiendas una película?")
memory.chat_memory.add_ai_message("¡Claro! ¿Qué género prefieres?")
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hola, ¿me recomiendas una película?'),
  AIMessage(content='¡Claro! ¿Qué género prefieres?')]}

#### ***Ejemplo: Usando `ConversationBufferWindowMemory`***

`ConversationBufferWindowMemory` en LangChain mejora las capacidades de manejo de memoria en aplicaciones conversacionales mediante la implementación de una ventana deslizante. Está diseñada para retener únicamente las últimas $k$ interacciones de la conversación, lo que permite limitar el tamaño del búfer y gestionar de manera más eficiente el contexto conversacional. Al utilizar sólo las interacciones recientes, se ayuda a reducir el número de tokens procesados, optimizando el rendimiento sin comprometer significativamente la relevancia del contexto.

Claves:

1. ***Ventana Deslizante:*** Almacena solo las últimas $k$ interacciones, proporcionando un enfoque práctico para manejar conversaciones largas sin sobrecargar la memoria del sistema.
2. ***Pérdida de Contexto:*** Aunque este enfoque puede perder el contexto de interacciones que preceden a las últimas K entradas, es un compromiso útil para mantener la eficiencia en conversaciones extensas o altamente dinámicas.
3. ***Gestión de Tokens:*** Reduce la carga computacional al limitar el número de tokens que el modelo de lenguaje necesita procesar en cada interacción.

***Chatbot de Soporte Técnico.***

Supongamos que estamos desarrollando un chatbot de soporte técnico que utiliza `ConversationBufferWindowMemory` para gestionar las interacciones con los usuarios. Este bot ayuda a los usuarios a solucionar problemas técnicos, y es crucial que recuerde las interacciones recientes para proporcionar soluciones coherentes sin retener toda la historia de la conversación.

In [5]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=2, return_messages=True)

## Otra manera de introduccir los mensajes a la memoria.
memory.save_context({"input": "Mi computadora no enciende."}, {"output": "¿Has intentado conectarla a otra toma de corriente?"})

memory.chat_memory.add_user_message("Sí, y sigue sin funcionar.")
memory.chat_memory.add_ai_message("¿Puedes verificar si la luz del cable de alimentación está encendida?")
memory.chat_memory.add_user_message("La luz está apagada.")
memory.chat_memory.add_ai_message("Parece que el problema podría estar en el cable de alimentación. ¿Tienes otro que puedas probar?")

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Sí, y sigue sin funcionar.'),
  AIMessage(content='¿Puedes verificar si la luz del cable de alimentación está encendida?'),
  HumanMessage(content='La luz está apagada.'),
  AIMessage(content='Parece que el problema podría estar en el cable de alimentación. ¿Tienes otro que puedas probar?')]}

#### ***Ejemplo: `ConversationSummaryMemory`***

Es un tipo de memoria más avanzado en LangChain diseñado para aplicaciones conversacionales que requieren la condensación de información a lo largo del tiempo. Esta memoria es especialmente útil para conversaciones prolongadas donde mantener un historial completo de mensajes en la entrada del modelo ocuparía demasiados tokens. `ConversationSummaryMemory` genera un resumen continuo de la conversación, almacenando y actualizando un sumario que puede ser inyectado en los prompts o cadenas para influir en las respuestas subsiguientes. Esta capacidad de resumir permite mantener relevante y compacto el contexto conversacional, optimizando el uso de recursos computacionales y mejorando la coherencia de las interacciones.

Características:

1. ***Generación de Resumen Dinámico:*** Automáticamente resume la conversación a medida que ocurre, proporcionando un contexto condensado pero comprensivo.
2. ***Optimización del Uso de Tokens:*** Al reducir la longitud del contexto que se pasa a los modelos de lenguaje, se minimiza el número de tokens necesarios, lo cual es crucial para limitar los costos y mejorar la eficiencia en modelos basados en tokens.
3. ***Aplicabilidad en Conversaciones Largas:*** Ideal para diálogos extendidos donde retener cada mensaje sería impracticable debido a las restricciones de longitud de entrada de los modelos de lenguaje modernos.

## ***Ejemplo: Implementación de memoria con LangChain y ChatOpenAI***

In [6]:
# Definimos una variable de entorno para la API KEY de OpenAI.
import os
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Eres un buen chatbot teniendo una conversación con un humano. Debes recordar el nombre del humano si es dado."
        ),
        HumanMessagePromptTemplate.from_template("{user_message}")
    ]
)

llm = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
)


llm.invoke({"user_message": "Hola me llamo Andres Campos, como estas?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un buen chatbot teniendo una conversación con un humano. Debes recordar el nombre del humano si es dado.
Human: Hola me llamo Andres Campos, como estas?

> Finished chain.


{'user_message': 'Hola me llamo Andres Campos, como estas?',
 'text': 'Hola, Andrés Campos. ¡Yo estoy bien, gracias! ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?'}

In [8]:
# No tiene memoria!.
llm.invoke("¿Cual es mi nombre?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un buen chatbot teniendo una conversación con un humano. Debes recordar el nombre del humano si es dado.
Human: ¿Cual es mi nombre?

> Finished chain.


{'user_message': '¿Cual es mi nombre?',
 'text': 'Lo siento, pero como soy un programa de inteligencia artificial, no tengo la capacidad de recordar información personal sobre los usuarios. ¿Hay algo más en lo que pueda ayudarte?'}

### ***Implementación de memoria con LangChain.***

In [9]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [10]:
# Ahora con memoria.
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Eres un buen chatbot teniendo una conversación con un humano."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{user_message}")
    ]
)

# Observe que `return_messages=True` y `"chat_history"` encaja en MessagesPlaceholder via `variable_name=chat_history`
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [11]:
conversation.invoke({"user_message": "Hola, me llamo Andres Campos, Me recomiendas una película por favor."})



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un buen chatbot teniendo una conversación con un humano.
Human: Hola, me llamo Andres Campos, Me recomiendas una película por favor.

> Finished chain.


{'user_message': 'Hola, me llamo Andres Campos, Me recomiendas una película por favor.',
 'chat_history': [HumanMessage(content='Hola, me llamo Andres Campos, Me recomiendas una película por favor.'),
  AIMessage(content='¡Hola, Andrés! Claro que sí, ¿qué tipo de películas te gustan? ¿Prefieres algún género en particular o tienes alguna película favorita que te pueda ayudar a recomendarte algo similar? ¡Cuéntame un poco más para poder sugerirte una película adecuada para ti!')],
 'text': '¡Hola, Andrés! Claro que sí, ¿qué tipo de películas te gustan? ¿Prefieres algún género en particular o tienes alguna película favorita que te pueda ayudar a recomendarte algo similar? ¡Cuéntame un poco más para poder sugerirte una película adecuada para ti!'}

In [12]:
conversation.invoke({"user_message": "Me gustan las películas de ciencia ficción."})



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un buen chatbot teniendo una conversación con un humano.
Human: Hola, me llamo Andres Campos, Me recomiendas una película por favor.
AI: ¡Hola, Andrés! Claro que sí, ¿qué tipo de películas te gustan? ¿Prefieres algún género en particular o tienes alguna película favorita que te pueda ayudar a recomendarte algo similar? ¡Cuéntame un poco más para poder sugerirte una película adecuada para ti!
Human: Me gustan las películas de ciencia ficción.



> Finished chain.


{'user_message': 'Me gustan las películas de ciencia ficción.',
 'chat_history': [HumanMessage(content='Hola, me llamo Andres Campos, Me recomiendas una película por favor.'),
  AIMessage(content='¡Hola, Andrés! Claro que sí, ¿qué tipo de películas te gustan? ¿Prefieres algún género en particular o tienes alguna película favorita que te pueda ayudar a recomendarte algo similar? ¡Cuéntame un poco más para poder sugerirte una película adecuada para ti!'),
  HumanMessage(content='Me gustan las películas de ciencia ficción.'),
  AIMessage(content='¡Perfecto, las películas de ciencia ficción suelen ser muy interesantes! Te recomiendo la película "Interestelar" dirigida por Christopher Nolan. Esta película combina elementos de ciencia ficción con drama y exploración espacial de una manera muy intrigante. La trama gira en torno a un grupo de astronautas que viajan a través de un agujero de gusano en busca de un nuevo hogar para la humanidad. ¡Es una película visualmente impresionante y con un

In [13]:
conversation.invoke(
    {
        "user_message": """Cual fue mi primera pregunta? Cual es mi nombre? Cual mi género favorito?
        Dame tu respuesta en bullets."""
    }
)



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un buen chatbot teniendo una conversación con un humano.
Human: Hola, me llamo Andres Campos, Me recomiendas una película por favor.
AI: ¡Hola, Andrés! Claro que sí, ¿qué tipo de películas te gustan? ¿Prefieres algún género en particular o tienes alguna película favorita que te pueda ayudar a recomendarte algo similar? ¡Cuéntame un poco más para poder sugerirte una película adecuada para ti!
Human: Me gustan las películas de ciencia ficción.
AI: ¡Perfecto, las películas de ciencia ficción suelen ser muy interesantes! Te recomiendo la película "Interestelar" dirigida por Christopher Nolan. Esta película combina elementos de ciencia ficción con drama y exploración espacial de una manera muy intrigante. La trama gira en torno a un grupo de astronautas que viajan a través de un agujero de gusano en busca de un nuevo hogar para la humanidad. ¡Es una película visualmente impresionante y con una historia cautivadora! Esp

{'user_message': 'Cual fue mi primera pregunta? Cual es mi nombre? Cual mi género favorito?\n        Dame tu respuesta en bullets.',
 'chat_history': [HumanMessage(content='Hola, me llamo Andres Campos, Me recomiendas una película por favor.'),
  AIMessage(content='¡Hola, Andrés! Claro que sí, ¿qué tipo de películas te gustan? ¿Prefieres algún género en particular o tienes alguna película favorita que te pueda ayudar a recomendarte algo similar? ¡Cuéntame un poco más para poder sugerirte una película adecuada para ti!'),
  HumanMessage(content='Me gustan las películas de ciencia ficción.'),
  AIMessage(content='¡Perfecto, las películas de ciencia ficción suelen ser muy interesantes! Te recomiendo la película "Interestelar" dirigida por Christopher Nolan. Esta película combina elementos de ciencia ficción con drama y exploración espacial de una manera muy intrigante. La trama gira en torno a un grupo de astronautas que viajan a través de un agujero de gusano en busca de un nuevo hogar p